### dependencies


In [3]:
import pandas as pd
import geopandas as gpd
import warnings
import topojson as tp
import requests
import os

# nationwide FIPS codes
fips_dict = {
    '01': 'AL',
    '02': 'AK',
    '04': 'AZ',
    '05': 'AR',
    '06': 'CA',
    '08': 'CO',
    '09': 'CT',
    '10': 'DE',
    '11': 'DC',
    '12': 'FL',
    '13': 'GA',
    '15': 'HI',
    '16': 'ID',
    '17': 'IL',
    '18': 'IN',
    '19': 'IA',
    '20': 'KS',
    '21': 'KY',
    '22': 'LA',
    '23': 'ME',
    '24': 'MD',
    '25': 'MA',
    '26': 'MI',
    '27': 'MN',
    '28': 'MS',
    '29': 'MO',
    '30': 'MT',
    '31': 'NE',
    '32': 'NV',
    '33': 'NH',
    '34': 'NJ',
    '35': 'NM',
    '36': 'NY',
    '37': 'NC',
    '38': 'ND',
    '39': 'OH',
    '40': 'OK',
    '41': 'OR',
    '42': 'PA',
    '44': 'RI',
    '45': 'SC',
    '46': 'SD',
    '47': 'TN',
    '48': 'TX',
    '49': 'UT',
    '50': 'VT',
    '51': 'VA',
    '53': 'WA',
    '54': 'WV',
    '55': 'WI',
    '56': 'WY'
}

url = 'https://transition.fcc.gov/oet/info/maps/census/fips/fips.txt'

# Fetch the content from the URL
response = requests.get(url)
response.raise_for_status()  # Check that the request was successful

table = response.text.split('------------    --------------\n')[1]

# Strip leading/trailing whitespace and split by newline
lines = table.strip().split('\n')

# Create a DataFrame from the list of lines
df = pd.DataFrame(lines, columns=['Data'])

# Split the 'Data' column on the first space
df[['FIPS', 'County_name']] = df['Data'].str.split(n=1, expand=True)

# Drop the original 'Data' column
df = df.drop(columns=['Data'])

# Drop rows where 'FIPS' ends with '000'
df = df[~df['FIPS'].str.endswith('000')]

# Extract the first 2 digits from 'FIPS' column
df['State_code'] = df['FIPS'].str[:2]

# Map 'State_code' to 'State' using fips_dict
df['State'] = df['State_code'].map(fips_dict)

# Drop the 'State_code' column if not needed
df = df.drop(columns=['State_code'])

df['county_state'] = df['County_name'] + ', ' + df['State']

# Create dictionary using zip and to_dict
nationwide_FIPSdict = dict(zip(df['FIPS'], df['county_state']))

nationwide_FIPSdict

{'01001': 'Autauga County, AL',
 '01003': 'Baldwin County, AL',
 '01005': 'Barbour County, AL',
 '01007': 'Bibb County, AL',
 '01009': 'Blount County, AL',
 '01011': 'Bullock County, AL',
 '01013': 'Butler County, AL',
 '01015': 'Calhoun County, AL',
 '01017': 'Chambers County, AL',
 '01019': 'Cherokee County, AL',
 '01021': 'Chilton County, AL',
 '01023': 'Choctaw County, AL',
 '01025': 'Clarke County, AL',
 '01027': 'Clay County, AL',
 '01029': 'Cleburne County, AL',
 '01031': 'Coffee County, AL',
 '01033': 'Colbert County, AL',
 '01035': 'Conecuh County, AL',
 '01037': 'Coosa County, AL',
 '01039': 'Covington County, AL',
 '01041': 'Crenshaw County, AL',
 '01043': 'Cullman County, AL',
 '01045': 'Dale County, AL',
 '01047': 'Dallas County, AL',
 '01049': 'DeKalb County, AL',
 '01051': 'Elmore County, AL',
 '01053': 'Escambia County, AL',
 '01055': 'Etowah County, AL',
 '01057': 'Fayette County, AL',
 '01059': 'Franklin County, AL',
 '01061': 'Geneva County, AL',
 '01063': 'Greene Co

### simplify tracts, derive counties


In [45]:
# ignore the warnings that come with simplifying geographically
warnings.filterwarnings("ignore", category=RuntimeWarning)

# simplify tracts --------------------------------------
tracts = gpd.read_file('Unused/tract_outlines.gpkg')
tracts['FIPS'] = tracts['STATEFP'] + tracts['COUNTYFP']
tracts = tracts[[
    'FIPS',
    'GEOID',
    'geometry'
]]

tracts['county_name'] = tracts['FIPS'].map(nationwide_FIPSdict)

toposimplify_tracts = 0.001
tracts_simp = tp.Topology(tracts, toposimplify=toposimplify_tracts).to_gdf()
tracts_simp.to_file('Data/tracts_simp.gpkg')

# create the counties by dissolving the tracts on the FIPS column
counties = tracts.dissolve(by='FIPS').reset_index()
counties = counties.drop(columns='GEOID')
counties['county_name'] = counties['FIPS'].map(nationwide_FIPSdict)
counties = counties[[
    'FIPS',
    'county_name',
    'geometry'
]]

# export simplified geometry
counties.to_file('Data/counties_simp.gpkg')
print('export complete!')

export complete!


,geometry,FIPS,GEOID,county_name
0,"MULTIPOLYGON (((-87.05135 35.64563, -87.05589 ...",47119,47119010600,"Maury County, TN"
1,"MULTIPOLYGON (((-87.26540 35.71902, -87.25406 ...",47119,47119010100,"Maury County, TN"
2,"MULTIPOLYGON (((-87.03520 35.60588, -87.04088 ...",47119,47119010500,"Maury County, TN"
3,"MULTIPOLYGON (((-87.09812 35.51216, -87.10095 ...",47119,47119011200,"Maury County, TN"
4,"MULTIPOLYGON (((-87.33923 35.65911, -87.33966 ...",47119,47119010900,"Maury County, TN"
...,...,...,...,...
429,"MULTIPOLYGON (((-86.62347 36.28037, -86.61356 ...",47165,47165021107,"Sumner County, TN"
430,"MULTIPOLYGON (((-86.59766 36.32375, -86.58041 ...",47165,47165021008,"Sumner County, TN"
431,"MULTIPOLYGON (((-86.64512 36.48743, -86.64604 ...",47165,47165020405,"Sumner County, TN"
432,"MULTIPOLYGON (((-86.56579 36.42511, -86.56685 ...",47165,47165020502,"Sumner County, TN"


### Convert STDB Excel files to CSV


In [4]:
# Need to open each Excel file downloaded from STDB, make a small change, and save
# Then run this script
def convert_excel_to_csv(directory, output_directory):
    for filename in os.listdir(directory):
        if filename.startswith("Color-coded maps") and filename.endswith(".xlsx"):
            # Construct the full path to the Excel file
            excel_path = os.path.join(directory, filename)

            # Read the Excel file into a DataFrame
            df = pd.read_excel(excel_path, engine='openpyxl')

            # Ensure the "Census Tract" column is of type object (string)
            df['Census Tract'] = df['Census Tract'].astype(str)

            # Rename the "Census Tract" column to "GEOID"
            df.rename(columns={'Census Tract': 'GEOID'}, inplace=True)

            # Construct the full path for the output CSV file
            csv_filename = filename.replace(".xlsx", ".csv")
            csv_path = os.path.join(output_directory, csv_filename)

            # Save the DataFrame to a CSV file
            df.to_csv(csv_path, index=False)
            print(f"Converted {filename} to {csv_filename}")


convert_excel_to_csv('Data/', 'Data/CSV/')

Converted Color-coded maps - 2024-2029 Growth Rate Population.xlsx to Color-coded maps - 2024-2029 Growth Rate Population.csv
Converted Color-coded maps - 2024 Senior Population.xlsx to Color-coded maps - 2024 Senior Population.csv
Converted Color-coded maps - 2024 Population Density.xlsx to Color-coded maps - 2024 Population Density.csv
Converted Color-coded maps - 2029 Total Population.xlsx to Color-coded maps - 2029 Total Population.csv
Converted Color-coded maps - 2024-2029 Growth Rate Owner Occ HUs.xlsx to Color-coded maps - 2024-2029 Growth Rate Owner Occ HUs.csv
Converted Color-coded maps - 2024 Total Population.xlsx to Color-coded maps - 2024 Total Population.csv
Converted Color-coded maps - 2024 Median Household Income.xlsx to Color-coded maps - 2024 Median Household Income.csv


In [54]:
gdf = gpd.read_file('Data/counties_simp.gpkg')

gdf

,FIPS,county_name,county_stripped,geometry
0,47015,"Cannon County, TN",Cannon,"POLYGON ((-85.93284 35.79853, -85.93680 35.793..."
1,47021,"Cheatham County, TN",Cheatham,"POLYGON ((-87.18227 36.05112, -87.18198 36.052..."
2,47037,"Davidson County, TN",Davidson,"MULTIPOLYGON (((-87.05341 36.04973, -87.05316 ..."
3,47043,"Dickson County, TN",Dickson,"MULTIPOLYGON (((-87.31521 35.96994, -87.31602 ..."
4,47081,"Hickman County, TN",Hickman,"POLYGON ((-87.26503 35.71857, -87.26354 35.717..."
5,47111,"Macon County, TN",Macon,"POLYGON ((-85.92066 36.62600, -85.91950 36.625..."
6,47119,"Maury County, TN",Maury,"POLYGON ((-87.09449 35.44209, -87.09445 35.442..."
7,47147,"Robertson County, TN",Robertson,"POLYGON ((-86.89896 36.38997, -86.89913 36.389..."
8,47149,"Rutherford County, TN",Rutherford,"POLYGON ((-86.69973 35.72561, -86.69961 35.726..."
9,47159,"Smith County, TN",Smith,"POLYGON ((-85.84352 36.28592, -85.84317 36.285..."
